In [1]:
import os
try:
    os.mkdir('centralities')
except:
    pass

In [2]:
fb = open('facebook_combined.txt', 'r')
nodes = set()
lines = fb.readlines()
for line in lines:
    node1, node2 = [int(a) for a in line.strip().split()]
    nodes.add(node1)
    nodes.add(node2)

In [3]:
nodes = list(nodes)
nodes.sort()
adjacency_list = [[] for node in nodes]
for line in lines:
    node1, node2 = [int(a) for a in line.strip().split()]
    adjacency_list[node1].append(node2)
    adjacency_list[node2].append(node1)

In [4]:
num = len(nodes)
sum_distance = [0 for node in nodes]
for node in nodes:
    visited = [0 for i in range(num)]
    visited[node] = 1
    all_nodes = []
    all_nodes.append((node, 0))
    while len(all_nodes) != 0:
        current_node, dist = all_nodes.pop(0)
        for NId in adjacency_list[current_node]:
            if visited[NId] == 0:
                all_nodes.append((NId, dist + 1))
                sum_distance[node] += (dist + 1)
                visited[NId] = 1

In [5]:
close = []
for node, value in enumerate(sum_distance):
    close.append((node, value))
close.sort(key=lambda x: x[1])

In [6]:
closeness = open('centralities/closeness.txt', 'w')
num = len(nodes)
for (node, value) in close:
    if value != 0:
        closeness.write(f'{node} {round((num - 1) / value, 6)}\n')
    else:
        closeness.write(f'{node} {0}\n')
closeness.close()

In [7]:
# using Brandes' Algorithm
betweenness = [0 for node in nodes]
for node in nodes:
    stack = []
    paths = [[] for n in nodes]
    number_of_paths = [0 for n in nodes]
    number_of_paths[node] = 1
    distances = [-1 for n in nodes]
    distances[node] = 0
    queue = []
    queue.append(node)
    while len(queue) > 0:
        vertex = queue.pop(0)
        stack.append(vertex)
        for node2 in adjacency_list[vertex]:
            if distances[node2] < 0:
                queue.append(node2)
                distances[node2] = distances[vertex] + 1
            if distances[node2] == distances[vertex] + 1:
                number_of_paths[node2] += number_of_paths[vertex]
                paths[node2].append(vertex)
    values = [0 for n in nodes]
    while len(stack) > 0:
        node2 = stack.pop()
        for vertex in paths[node2]:
            values[vertex] += (number_of_paths[vertex] / number_of_paths[node2]) * (1 + values[node2])
        if node2 != node:
            betweenness[node2] += values[node2]

In [8]:
between = []
for i in range(len(betweenness)):
    betweenness[i] /= ((num - 1) * (num - 2))
    between.append((i, betweenness[i]))

In [9]:
between.sort(reverse=True, key=lambda x: x[1])

In [10]:
betweennesses = open('centralities/betweenness.txt', 'w')
for node, value in between:
    betweennesses.write(f'{node} ')
    betweennesses.write('%.6f' % round(value, 6))
    betweennesses.write('\n')
betweennesses.close()

In [11]:
div_4 = 0
for node in nodes:
    if node % 4 == 0:
        div_4 += 1
d = [(1 / div_4) if node % 4 == 0 else 0 for node in nodes]
page_ranks = d[:]
new_page_ranks = [0 for i in page_ranks]
for i in range(100):
    for node in nodes:
        t = 0
        for node2 in adjacency_list[node]:
            t = t + page_ranks[node2] / len(adjacency_list[node2])
        new_page_ranks[node] = 0.8 * t + 0.2 * d[node]
    sum_l1 = sum(new_page_ranks)
    for j in range(len(new_page_ranks)):
        new_page_ranks[j] /= sum_l1
    loop_again = False
    for j in range(len(page_ranks)):
        if abs(page_ranks[j] - new_page_ranks[j]) > 1e-8:
            loop_again = True
            break
    if not loop_again:
#         print(i + 1, "iterations")
        break
    page_ranks = new_page_ranks[:]

In [12]:
for node, rank in enumerate(page_ranks):
    page_ranks[node] = (node, rank)
page_ranks.sort(reverse=True, key=lambda x: x[1])

In [13]:
page_rank_centralities = open('centralities/pagerank.txt', 'w')
for node, rank in page_ranks:
    page_rank_centralities.write(f'{node} ')
    page_rank_centralities.write('%.6f\n' % round(rank, 6))
page_rank_centralities.close()

In [14]:
import snap

In [15]:
fb = open('facebook_combined.txt', 'r')
G = snap.TUNGraph().New()
lines = fb.readlines()
for line in lines:
    node1, node2 = [int(a) for a in line.strip().split()]
    try:
        G.AddNode(node1)
    except:
        pass
    try:
        G.AddNode(node2)
    except:
        pass
    G.AddEdge(node1, node2)

In [16]:
snap_closeness = []
for node in snap.Nodes(G):
    value = snap.GetClosenessCentr(G, node.GetId())
    snap_closeness.append((node.GetId(), value))
snap_closeness.sort(reverse=True, key=lambda x: x[1])

In [17]:
num = G.GetNodes()

In [18]:
snap_between = []
Nodes = snap.TIntFltH()
Edges = snap.TIntPrFltH()
snap.GetBetweennessCentr(G, Nodes, Edges, 0.8)
for node in Nodes:
    snap_between.append((node, 2 * Nodes[node] / ((num - 1) * (num - 2))))
snap_between.sort(reverse=True, key=lambda x: x[1])

In [19]:
snap_pagerank = []
PRankH = snap.TIntFltH()
snap.GetPageRank(G, PRankH, 0.8, 1e-8)
for node in PRankH:
    snap_pagerank.append((node, PRankH[node]))
snap_pagerank.sort(reverse=True, key=lambda x: x[1])

In [20]:
closeness = open('centralities/closeness.txt', 'r')
betweenness = open('centralities/betweenness.txt', 'r')
pagerank = open('centralities/pagerank.txt', 'r')
nodes_close = set()
nodes_between = set()
nodes_pr = set()
snap_nodes_close = set()
snap_nodes_between = set()
snap_nodes_pr = set()
for i in range(100):
    nodes_close.add(int(closeness.readline().strip().split()[0]))
    nodes_between.add(int(betweenness.readline().strip().split()[0]))
    nodes_pr.add(int(pagerank.readline().strip().split()[0]))
    snap_nodes_close.add(snap_closeness[i][0])
    snap_nodes_between.add(snap_between[i][0])
    snap_nodes_pr.add(snap_pagerank[i][0])

In [21]:
print(f'#overlaps for Closeness Centrality: {len(nodes_close.intersection(snap_nodes_close))}')
print(f'#overlaps for Betweenness Centrality: {len(nodes_between.intersection(snap_nodes_between))}')
print(f'#overlaps for Closeness Centrality: {len(nodes_pr.intersection(snap_nodes_pr))}')

#overlaps for Closeness Centrality: 100
#overlaps for Betweenness Centrality: 97
#overlaps for Closeness Centrality: 60
